# Object Detection Training and Evaluation Pipeline

In [1]:
# will run necessary scrpts through the os.system method

import os, subprocess

# we can run everything based on the assumed folder structure
# add folder structure here #

# the name of the project
project_name = 'vehicle_detection_in_satellite_imagery'

#class model:
#    def __init__(name, details):

# dict of competing models

competing_faster_rcnn_resnet_models = {"1a": {"name": "50deep_640by640_1", 
                                              "exported_model_name": "1a",
                                              "details": "trained for only 25k iterations"
                                             },
                                       "1b": {"name": "50deep_640by640_2", 
                                              "exported_model_name": "1b",
                                              "details": "trained for 85k iterations"
                                             },
                                       "2a": {"name": "101deep_640by640_1", 
                                              "exported_model_name": "2a",
                                              "details": "trying a larger model", 
                                              "results": "oscilated loss attribued to high learning rate"
                                             },
                                       "2b": {"name": "101deep_640by640_2", 
                                              "exported_model_name": "2b",
                                              "details": "trying lower learning rate", 
                                              "results": "persisting oscilaion"},
                                       "2c": {"name": "101deep_640by640_3", 
                                              "exported_model_name": "2c",
                                              "details": "much lower learning rate", 
                                              "result": "loss oscilation"},
                                       "2d": {"name": "101deep_640by640_4", 
                                              "exported_model_name": "2d",
                                              "details": "order 10 lower learning rate and .99 momentum", 
                                              "results": "no convergence although less oscilation"
                                             },
                                       "1c": {"name": "50deep_640by640_3", 
                                              "exported_model_name": "1c",
                                              "details": "increasing first_stage_max_proposals from 300 to 600, only run for 40k steps",
                                              "results": "didn't seem to train that well, will experiment with lower learning rate"
                                             },
                                       "1d": {"name": "50deep_640by640_4", 
                                              "exported_model_name": "1d",
                                              "details": "increasing first_stage_max_proposals from 300 to 1k, with 50k steps 0.95 momentum and factor 10 lower learning rate",
                                              "results": "didn't seem to train that well, will experiment with lower learning rate"
                                             },
                                       "3a": {"name": "centernet_hrglass104_512", 
                                              "exported_model_name": "3a",
                                              "details": "according to model garden repo should obtain higher mAP (although with lower speed)",
                                              "results": ""
                                             },
                                       "3e": {"name": "centernet_hrglass104_512_5", 
                                              "exported_model_name": "3e",
                                              "details": "as per 3d except double the batch size",
                                              "results": "cut it short, so only went to about 25k - "
                                             },
                                       "3c": {"name": "centernet_hrglass104_512_3", 
                                              "exported_model_name": "3c",
                                              "details": "reducing training time to 40k and introducing data augmentation\
                                              to improve applicability to images - also reduced batch size to 4 (no delta adjustment)",
                                              "results": "very good pickup of cars, though a lot of false positives"
                                             },
                                       "3d": {"name": "centernet_hrglass104_512_4", 
                                              "exported_model_name": "3d",
                                              "details": "as per 3d except with double the batch size (8) and lower delta (factor of 10 lower)",
                                              "results": ""
                                             },
                                       "3f": {"name": "centernet_hrglass104_512_6", 
                                              "exported_model_name": "3f",
                                              "details": "Seeing what happens if I drop down to 12k steps. Also removing the augmentation options\
                                              other than the original ones",
                                              "results": "cancelled this one"
                                             },
}

#TODO: make it work off a list, at the moment its just a single model
model_working_with = "3e"                                      
chosen_model = competing_faster_rcnn_resnet_models[model_working_with]["name"]


## Imagery Preprocessing

### Splitting into test and train

In [2]:
#set working drecory that of the preprocessing scripts
working_directory = r"C:/Users/paperspace/Documents/Tensorflow/workspace/{0}/scripts/preprocessing".format(project_name)
#print(working_directory)
os.chdir(working_directory) 

# running the script to partition the dataset, it has arguments:
#python partition_dataset.py 
# --imageDir "C:/Users/paperspace/Documents/Tensorflow/workspace/object_detection_cats_in_gardens/images" 
# --xml

image_directory = "C:/Users/paperspace/Documents/Tensorflow/workspace/{0}/images".format(project_name)
os.system("start cmd")
partion_dataset_command = """python partition_dataset.py --imageDir "{0}"  --xml""".format(image_directory)
print("run this: \n {0}".format(partion_dataset_command))

run this: 
 python partition_dataset.py --imageDir "C:/Users/paperspace/Documents/Tensorflow/workspace/vehicle_detection_in_satellite_imagery/images"  --xml


### Creating the label map

Creating this (or rather, forgetting to create it) caused me significant problems during the 

"""
item {
    id: 1
    name: 'cat'
}
"""

In [3]:
#write the label map to a file

class_names = ['class1', 'class2', 'etc']

label_map_text =\
"""item {
  id: 1
  name: 'car'
}
item {
  id: 2
  name: 'boat'
}
item {
  id: 3
  name: 'camping car'
}
item {
  id: 4
  name: 'pickup'
}
item {
  id: 5
  name: 'plane'
}
item {
  id: 6
  name: 'tractor'
}
item {
  id: 7
  name: 'truck'
}
item {
  id: 8
  name: 'van'
}
item {
  id: 9
  name: 'other'
}
item {
  id: 10
  name: 'motorcycle'
}
"""

label_map_file_directory = "C:/Users/paperspace/Documents/Tensorflow/workspace/{0}/annotations/label_map.pbtxt".format(project_name)

label_map_file_writer = open(label_map_file_directory,"w")

for line in label_map_text.splitlines():
    label_map_file_writer.writelines(line+"\n")
    
label_map_file_writer.close()


### Creating the tf records and protobufs for training

In [4]:
#set working drecory that of the preprocessing scripts
working_directory = r"C:/Users/paperspace/Documents/Tensorflow/workspace/{0}/scripts/preprocessing".format(project_name)
#print(working_directory)
os.chdir(working_directory) 

# running the script to generate tfrecords, it has arguments:
#for testing:
#python generate_tfrecord.py 
# --xml_dir = [where xml files are stored]
# --labels_path 
# --output_path 
# --csv_path=[OPTIONAL] 

# putting the required info into a dict 
inputs = {}
for genre in ["test", "train"]:
    inputs[genre] = {}
    inputs[genre]["xml_directory"] = "C:/Users/paperspace/Documents/Tensorflow/workspace/{0}/images/{1}".format(project_name, genre)
    inputs[genre]["labels_directory"] = "C:/Users/paperspace/Documents/Tensorflow/workspace/{0}/annotations/label_map.pbtxt".format(project_name)
    inputs[genre]["output_directory"] = "C:/Users/paperspace/Documents/Tensorflow/workspace/{0}/annotations/{1}.record".format(project_name, genre)

# start the cmd and set the directory and then give the required inputs for manual execustion
#TODO: generate a new cmd window and run things while the user just watches
os.system("start cmd")
train_tfrecords_command = """python generate_tfrecord.py --xml_dir "{0}" --labels_path {1} --output_path {2} """.format(inputs["train"]["xml_directory"], inputs["train"]["labels_directory"], inputs["train"]["output_directory"])
test_tfrecords_command =  """python generate_tfrecord.py --xml_dir "{0}" --labels_path {1} --output_path {2} """.format(inputs["test"]["xml_directory"],inputs["test"]["labels_directory"],inputs["test"]["output_directory"])

print("---Train---\nrun this: \n {0}".format(train_tfrecords_command))
print("\n---Test---\nrun this: \n {0}".format(test_tfrecords_command))

---Train---
run this: 
 python generate_tfrecord.py --xml_dir "C:/Users/paperspace/Documents/Tensorflow/workspace/vehicle_detection_in_satellite_imagery/images/train" --labels_path C:/Users/paperspace/Documents/Tensorflow/workspace/vehicle_detection_in_satellite_imagery/annotations/label_map.pbtxt --output_path C:/Users/paperspace/Documents/Tensorflow/workspace/vehicle_detection_in_satellite_imagery/annotations/train.record 

---Test---
run this: 
 python generate_tfrecord.py --xml_dir "C:/Users/paperspace/Documents/Tensorflow/workspace/vehicle_detection_in_satellite_imagery/images/test" --labels_path C:/Users/paperspace/Documents/Tensorflow/workspace/vehicle_detection_in_satellite_imagery/annotations/label_map.pbtxt --output_path C:/Users/paperspace/Documents/Tensorflow/workspace/vehicle_detection_in_satellite_imagery/annotations/test.record 


### setting up a pipeline config file based on a model selection

For the moment we assume the model directory and the pipelie is already selected

Lets jsut provide a nice list of text to copy into the config file for the chosen model


In [5]:
print("---- under train_input_reader ----")
print("set the label_map_path to: ")
print("C:/Users/paperspace/Documents/Tensorflow/workspace/{0}/annotations/label_map.pbtxt".format(project_name))

print("set the input_path to: ")
print("C:/Users/paperspace/Documents/Tensorflow/workspace/{0}/annotations/train.record".format(project_name))

print("---- under eval_input_reader ----")

print("set the label_map_path to: ")
print("C:/Users/paperspace/Documents/Tensorflow/workspace/{0}/annotations/label_map.pbtxt".format(project_name))

print("set the input_path to: ")
print("C:/Users/paperspace/Documents/Tensorflow/workspace/{0}/annotations/test.record".format(project_name))

print("don't forget to set the batch sizes and the option which only weks for TPUs and the number of classes - and comment out checkpoint settings")

---- under train_input_reader ----
set the label_map_path to: 
C:/Users/paperspace/Documents/Tensorflow/workspace/vehicle_detection_in_satellite_imagery/annotations/label_map.pbtxt
set the input_path to: 
C:/Users/paperspace/Documents/Tensorflow/workspace/vehicle_detection_in_satellite_imagery/annotations/train.record
---- under eval_input_reader ----
set the label_map_path to: 
C:/Users/paperspace/Documents/Tensorflow/workspace/vehicle_detection_in_satellite_imagery/annotations/label_map.pbtxt
set the input_path to: 
C:/Users/paperspace/Documents/Tensorflow/workspace/vehicle_detection_in_satellite_imagery/annotations/test.record
don't forget to set the batch sizes and the option which only weks for TPUs and the number of classes - and comment out checkpoint settings


### Runnng the script to train the model 

In [6]:
#set working drecory that of the scripts and open ready to go
working_directory = r"C:/Users/paperspace/Documents/Tensorflow/workspace/{0}/scripts".format(project_name)
os.chdir(working_directory)
os.system("start cmd")
#TODO: unify use of directories in this script
config_directory = r"C:\Users\paperspace\Documents\Tensorflow\workspace\{0}\models\{1}\pipeline.config".format(project_name,chosen_model)

model_directory = r"C:\Users\paperspace\Documents\Tensorflow\workspace\{0}\models\{1}".format(project_name,chosen_model)

print("run the following")
print('python model_main_tf2.py --pipeline_config_path=\"{0}\" --model_dir=\"{1}\" '.format(config_directory, model_directory))


run the following
python model_main_tf2.py --pipeline_config_path="C:\Users\paperspace\Documents\Tensorflow\workspace\vehicle_detection_in_satellite_imagery\models\centernet_hrglass104_512_5\pipeline.config" --model_dir="C:\Users\paperspace\Documents\Tensorflow\workspace\vehicle_detection_in_satellite_imagery\models\centernet_hrglass104_512_5" 


### Reviewing model training progress with Tensorboard 

In [7]:
#set working drecory that of the scripts and open ready to go
working_directory = r"C:/Users/paperspace/Documents/Tensorflow/workspace/{0}/scripts".format(project_name)
os.chdir(working_directory)
os.system("start cmd")

#generate and print the command to get it started
tensorboard_command = 'tensorboard --logdir=\"{0}\"'.format(model_directory)
print("run the following")
print(tensorboard_command)

run the following
tensorboard --logdir="C:\Users\paperspace\Documents\Tensorflow\workspace\vehicle_detection_in_satellite_imagery\models\centernet_hrglass104_512_5"


### Exporting the Model 

In [8]:
# set a name for the exported model, note it must be different to the 
#exported_model_name = "frcn5064016100"
exported_model_name = competing_faster_rcnn_resnet_models[model_working_with]["exported_model_name"]

#set working drecory that of the scripts and open ready to go
working_directory = r"C:/Users/paperspace/Documents/Tensorflow/workspace/{0}/scripts".format(project_name)
os.chdir(working_directory)
os.system("start cmd")

#TODO: unify the paths used in the model in a single dict and then write to it during

pipeline_config_path = r"C:\Users\paperspace\Documents\Tensorflow\workspace\{0}\models\{1}\pipeline.config".format(project_name, chosen_model)
trained_checkpoint_directory = r"C:\Users\paperspace\Documents\Tensorflow\workspace\{0}\models\{1}".format(project_name, chosen_model)
output_directory = r"C:\Users\paperspace\Documents\Tensorflow\workspace\{0}\exported-models\{1}".format(project_name, exported_model_name)

#python exporter_main_v2.py --input_type image_tensor 
#--pipeline_config_path pipeline_config_path
#--trained_checkpoint_dir trained_checkpoint_dir
#--output_directory output_directory


model_export_command = 'python exporter_main_v2.py --input_type image_tensor --pipeline_config_path "{0}" --trained_checkpoint_dir "{1}\" --output_directory "{2}"'.format(pipeline_config_path, trained_checkpoint_directory, output_directory)

print("use the following command \n")
print(model_export_command)

use the following command 

python exporter_main_v2.py --input_type image_tensor --pipeline_config_path "C:\Users\paperspace\Documents\Tensorflow\workspace\vehicle_detection_in_satellite_imagery\models\centernet_hrglass104_512_5\pipeline.config" --trained_checkpoint_dir "C:\Users\paperspace\Documents\Tensorflow\workspace\vehicle_detection_in_satellite_imagery\models\centernet_hrglass104_512_5" --output_directory "C:\Users\paperspace\Documents\Tensorflow\workspace\vehicle_detection_in_satellite_imagery\exported-models\3e"


In [9]:
### Evaluating the Model

# Testing the image processing tool:

We test the image processing tool with the demonstration image contained in the same directory as this notebook.

In [12]:
%matplotlib inline

In [13]:
import time
import tensorflow as tf

# get the location of the model
PATH_TO_SAVED_MODEL = output_directory+"\saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...WARNING:tensorflow:Importing a function (__inference_batchnorm_layer_call_and_return_conditional_losses_53828) with ops with custom gradients. Will likely fail if a gradient is requested.


Done! Took 91.61676049232483 seconds


In [ ]:
import numpy as np
import cv2
import math
import copy
import os
from operator import itemgetter, attrgetter
from matplotlib.pyplot import subplots
from statistics import median


class detection:
    """
    """
    def __init__(self, box_array, score, class_no, detection_number, parent_snippet_y_min, parent_snippet_x_min, parent_snippet_size):
        #set box limits 
        ymin_ratio_np, xmin_ratio_np, ymax_ratio_np, xmax_ratio_np = box_array
        self.ymin_ratio, self.xmin_ratio, self.ymax_ratio, self.xmax_ratio = float(ymin_ratio_np), float(xmin_ratio_np), float(ymax_ratio_np), float(xmax_ratio_np)
        
        #set the score and the class number
        self.score = float(score.numpy()) #TODO: do something clean
        self.class_no = float(class_no.numpy()) #TODO: do something clean
        # reverse score for sorting 
        self.detection_score_sort = 1.0-score
        # convert the local coordinates within the snippet (ratios) to clobal image coordinates
        
        self.y_min, self.x_min, self.y_max, self.x_max = self.find_global_coordinates(parent_snippet_y_min = parent_snippet_y_min, 
                                                                                        parent_snippet_x_min = parent_snippet_x_min, 
                                                                                        parent_snippet_size = parent_snippet_size
                                                                                        )
        
        # set the status of the detection to "active" as opposed to deleted when there is a matching item
        
        self.active = True
        
        #name is parent name plus the snippet_number
        
        #get the centroid of the bounidng box
        self.centroidx = (self.x_max + self.x_min)/2.0
        self.centroidy = (self.y_max + self.y_min)/2.0
        
        # get distance across the detection box
        
        self.quasi_radius = self.find_quasi_radius()
        
        # add the size of the box        
        self.size = (self.y_max - self.y_min)*(self.x_max - self.x_min)
        
        # set the id which references the objects position in the image list and which will be overwritten by the image method
        self.id = None
        
        self.possible_overlaps = []
        
        self.overlaps = []
    
    def identify_possible_overlaps(self, all_detections_in_image, num_quasi_radii):
        """Given a list of detection objects at the image level, this function identify those for which it is 
        worthwhile to check the intersection

        The rule is basically that if they are worth being checked the sum of the quasi-radii of the two 
        boxes will be less than the distance between them.
        """
        #TODO: come up with a more robust methodolgy than the differnce in x_mins vs radii shit
        possible_overlaps = [detection for detection in all_detections_in_image if detection.id!=self.id]
        
        possible_overlaps = [detection for detection in possible_overlaps if abs(detection.x_min-self.x_min)<=(self.quasi_radius+detection.quasi_radius)]
        
        self.possible_overlaps = [detection.id for detection in possible_overlaps if abs(detection.y_min-self.y_min)<=(self.quasi_radius+detection.quasi_radius)]
        
    def identify_overlaps(self, all_detections_in_image, intersection_over_union_cutoff):
        """
        Given the list of ids (that is ids in the original image which represent the location in the image's detections attributie (list)
        and the provided iou cutoff and preference for cars, we perform a 
        
        """
        
        for possible_overlap_id in self.possible_overlaps:
            #compare intersection over union (IOU) with cutoff and add to 
            other_detection = all_detections_in_image[possible_overlap_id]
            iou = self.calculate_IOU(other_detection)
            if iou>=intersection_over_union_cutoff:
                self.overlaps.append(other_detection.id)
                
    
    def find_quasi_radius(self):
        a_squared = (self.y_max - self.y_min)**2
        b_squared = (self.x_max - self.x_min)**2
        
        return math.sqrt(a_squared + b_squared)/2
        
    def find_global_coordinates(self, parent_snippet_y_min, parent_snippet_x_min, parent_snippet_size):
        """Function which finds the location of an object in the 
           scope of the original image, based on the relative coordinates
           provided in the 
        
        """
        x_offset = parent_snippet_x_min
        y_offset = parent_snippet_y_min
        
        y_min = y_offset + (self.ymin_ratio*parent_snippet_size)
        y_max = y_offset + (self.ymax_ratio*parent_snippet_size)
        x_min = x_offset + (self.xmin_ratio*parent_snippet_size)
        x_max = x_offset + (self.xmax_ratio*parent_snippet_size)
        
        return y_min, x_min, y_max, x_max
    
    def set_id(self, new_id):
        """
        Set an id corresponding to the location of the detection object in the 
        list of detection objects at the image level
        """
        self.id = new_id
    
    def calculate_IOU(self, other):
        """
        Calculates the IOU
        """
        
        intersection_y_max = min(self.y_max, other.y_max)
        intersection_x_max = min(self.x_max, other.x_max)
        intersection_y_min = max(self.y_min, other.y_min)
        intersection_x_min = max(self.x_min, other.x_min)
        
        intersection_size = (intersection_y_max - intersection_y_min)*(intersection_x_max - intersection_x_min)
        
        return intersection_size/(self.size + other.size - intersection_size)
    
    def print_bounds(self):
        print("y_min: {0}, x_min: {1}, y_max: {2}, x_max: {3}".format(round(self.y_min,1),
                                                                        round(self.x_min,1),
                                                                        round(self.y_max,1),
                                                                        round(self.x_max,1)
                                                                        ))

class snippet:
    """
    Attributes:
     - image_as_nparray
     - y_min
     - y_max
     - x_min
     - x_max
     - parent_image_name
     - <outdated>detections</outdated>

     
    Methods:
     - __init__(self, image_as_nparray, snippet_size, upper_limit, lower_limit, parent_image_name
    """
    def __init__(self, snippet_size, x_min, y_min, image_as_nparray, image_xmax,
                image_ymax, parent_image_name, is_offset):
        """
        """
        self.x_min = x_min
        self.y_min = y_min
        self.size = snippet_size
        self.parent_image_name = parent_image_name
        self.is_offset = is_offset
        
        self.x_max, self.y_max, self.padding_x, self.padding_y = self.get_appropriate_bounds(x_min = x_min, 
                                                                                            y_min = y_min, 
                                                                                            snippet_size = snippet_size, 
                                                                                            image_xmax = image_xmax, 
                                                                                            image_ymax = image_ymax
                                                                                            )
        
        self.name = "{0}_{1}_{2}_{3}".format(self.x_min, self.x_max, self.y_min, self.y_max) 
        if (self.padding_x==0) & (self.padding_y==0):
            self.image_as_nparray = image_as_nparray[self.y_min: self.y_max, self.x_min: self.x_max]
        else:
            self.image_as_nparray = cv2.copyMakeBorder(image_as_nparray[self.y_min: self.y_max, self.x_min: self.x_max], 
                                                        0, 
                                                        self.padding_y, 
                                                        0, 
                                                        self.padding_x, 
                                                        cv2.BORDER_CONSTANT
                                                        )
    
    def return_detected_objects(self, detection_function, score_cutoff):
        """
        GENERATEOR
        Detects the objects in the image snipet using the provided TF detection function
        and then converts each one to a detection object (through the init method of the 
        detection class). 
        ## update: the detections are no longer added to the snippet, they are just passed to image
        ####<Outdated>
        ##Each object detected (above the given confidence level cutoff) 
        #is added to the detections list attribute.
        ####</Outdated>
        
        In order to allow the comparison of detections across different snippets, in addition
        to writing the detecion objects to a list attribute in the sniipet, it also acts as a
        generator which yields each detectio object
        
        
        """
        
        #TODO: take advantage of the batch processing tensor function instead of just doing this:
        #reading just one image
        test_tensor = tf.convert_to_tensor(self.image_as_nparray)
        input_tensor = test_tensor[tf.newaxis, ...]
        
        #apply the model to run inference
        detections = detection_function(input_tensor)
        
        #for each potential detection in the result we generate a detection object named i
        i=0
        for box_limits, score, class_no in zip(detections["detection_boxes"][0], detections["detection_scores"][0], detections["detection_classes"][0]):
            if score>score_cutoff:
                detected_object = detection(box_array = box_limits,
                                            score = score, 
                                            class_no = class_no, 
                                            detection_number = i,
                                            parent_snippet_y_min = self.y_min, 
                                            parent_snippet_x_min = self.x_min, 
                                            parent_snippet_size = self.size
                                            )
                i+=1
                yield detected_object
    
    def get_appropriate_bounds(self, x_min, y_min, snippet_size, image_xmax, image_ymax):
        """
        need to identify the bounds of the wider image to pull and then padd 
        them with black as necessary to make all images the correct size
        """
        if (x_min + snippet_size > image_xmax) & (y_min + snippet_size > image_xmax):
            x_max, y_max = image_xmax, image_ymax
            padding_x = (x_min + snippet_size) - image_xmax
            padding_y = (y_min + snippet_size) - image_ymax
            
        elif (x_min + snippet_size > image_xmax):
            x_max, y_max = image_xmax, y_min + snippet_size
            padding_x = (x_min + snippet_size) - image_xmax
            padding_y = 0
            
        elif (y_min + snippet_size > image_xmax):
            x_max, y_max = x_min + snippet_size, image_ymax
            padding_x = 0
            padding_y = (y_min + snippet_size) - image_ymax
            
        else:
            x_max, y_max = x_min + snippet_size, y_min + snippet_size
            padding_x, padding_y = 0,0
        
        return x_max, y_max, padding_x, padding_y

class image:
    """
    Attributes:
    - image_path
    - image_name
    - store_number
    - month
    - year
    - image_as_nparray

    """ 
    def __init__(self, image_path, image_name="from_filename"):
        if image_name!="from_filename":
            self.image_name = image_name
        else:
            self.image_name = image_path.split("\\")[-1].split(".")[0]
        self.image_path = image_path
        
        # obtain store details and image date from the filename
        details = self.image_name.split("_")
        self.store_number = details[0]
        self.month = details[1]
        self.year = details[2]
        
        # pull in image as an nparray, get dimensions
        self.image_as_nparray = cv2.imread(image_path)
        self.y_max = self.image_as_nparray.shape[0]
        self.x_max = self.image_as_nparray.shape[1]
        
        # add empty lists for both image snippet objects and detections which will be added to by methods
        self.snippets = []
        self.detections = []
    
    def cut_image(self, output_size, xy_inialisation, x_max, y_max, is_offset):
        """
        """
        rows = 0
        images = 0
        #check inputs
        if (type(output_size) != int) | (type(x_max) != int) | (type(y_max) != int):
            print("{0}, {1}, {2} must be int".format("output_size" if (type(output_size) != int) else "",
                                                    "x_max" if (type(x_max) != int) else "",
                                                    "y_max" if (type(y_max) != int) else ""))
                 #we will scan across the image left to right then down
        y = xy_inialisation
        while (y_max-y)>=output_size:
            rows += 1
            x = xy_inialisation
            
            while (x_max-x)>=output_size:
                images += 1
                self.snippets.append(snippet(snippet_size = output_size, 
                                            x_min = x, 
                                            y_min = y, 
                                            image_as_nparray = self.image_as_nparray, 
                                            image_xmax = self.x_max,
                                            image_ymax = self.y_max, 
                                            parent_image_name = self.image_name,  
                                            is_offset = is_offset))
                x += output_size
            else:
                images += 1
                self.snippets.append(snippet(snippet_size = output_size, 
                                            x_min = x, 
                                            y_min = y, 
                                            image_as_nparray = self.image_as_nparray, 
                                            image_xmax = self.x_max,
                                            image_ymax = self.y_max, 
                                            parent_image_name = self.image_name,  
                                            is_offset = is_offset))
                x += output_size
            y += output_size
        else:
            rows += 1
            x = xy_inialisation
            
            while (x_max-x)>=output_size:
                images += 1
                self.snippets.append(snippet(snippet_size = output_size, 
                                            x_min = x, 
                                            y_min = y, 
                                            image_as_nparray = self.image_as_nparray, 
                                            image_xmax = self.x_max,
                                            image_ymax = self.y_max, 
                                            parent_image_name = self.image_name,  
                                            is_offset = is_offset))
                x += output_size
            else:
                images += 1
                self.snippets.append(snippet(snippet_size = output_size, 
                                            x_min = x, 
                                            y_min = y, 
                                            image_as_nparray = self.image_as_nparray, 
                                            image_xmax = self.x_max,
                                            image_ymax = self.y_max, 
                                            parent_image_name = self.image_name,  
                                            is_offset = is_offset))
                x += output_size
            y += output_size
        return int(rows), int(images/rows)
    
    def split_into_snippets(self, output_size, with_offsets):
        """
        output_size:    (int) the required height and width of the image
        with_offsets:   (bool) whether or not offsetting snippets i.e., 
                        images starting output_size/2 from the edge and 
                        finishing in the same place on the other side, top, bottom
        
        Returns:        list of snippet objects added to the snippets list attribute
        """
        #offset adjustment in case of splitting with offsets 
        offset_adjustment = (output_size)/2
        
        # add the cut images to the snippets attribute
        self.rows, self.columns = self.cut_image(output_size, 
                                                 0, 
                                                 self.x_max, 
                                                 self.y_max,
                                                 is_offset = False) #for the regular
        if with_offsets:#for the offsets
            self.offset_rows, self.offset_columns = self.cut_image(output_size, 
                                                                    int(offset_adjustment), 
                                                                    int(self.x_max - offset_adjustment), 
                                                                    int(self.y_max - offset_adjustment), #TODO: don't just be lazy and cast, work out why they're ints atm
                                                                    is_offset = True)
    def plot_image_snippets(self, fig_size):
        """
        """#TODO: split into two or give option to choose which lone to plot
        fig,axes = plt.subplots(nrows = self.rows, ncols = self.columns, figsize=(fig_size,fig_size))
        ##label as numbers for each image
        image_no=0
        for i in range(self.rows):
            for j in range(self.columns):
                axes[i,j].imshow(self.snippets[image_no].image_as_nparray)
                axes[i,j].axis('off')
                image_no += 1
        plt.show()
        
        fig,axes = plt.subplots(nrows = self.offset_rows, ncols = self.offset_columns, figsize=(fig_size,fig_size))
        image_no = self.rows*self.columns
        for i in range(self.offset_rows):
            for j in range(self.offset_columns):
                axes[i,j].imshow(self.snippets[image_no].image_as_nparray)
                axes[i,j].axis('off')
                image_no += 1
        plt.show()
    
    def detect_objects(self, detection_function, score_cuttoff):
        """
        Detects all objects in the snippets within this image
        """
        # set a detection_counter so that each detection can be referenced by its position in the image.detections list
        i=0
        for snippet in self.snippets:
            for detection in snippet.return_detected_objects(detection_function = detection_function, score_cutoff = score_cuttoff):
                self.detections.append(detection)
                detection.set_id(i)
                i+=1
    
    def remove_overlapping_objects(self, num_quasi_radii, intersection_over_union_cutoff, preference_cars = True):
        """
        Identifies detection objects with overlapping bounding boxes and then Sets the active attribute 
        of the detection(s) with lower score 
        Preferences detectuibs which are inferred to be cars regardless of score by default
        """
        
        #TODO: this is super messy, should be in sub functions, look into it.
        #it is important that the process of setting active = False is undertaken from the objects
        #with the highest scores first. However if I sort te objects it;s going to fuck up the system of refering
        #the objects based on their position in the detection lists. So instad we get a list of the ids in order
        # if there is a preference for cars this sorting needs to be first on the clas id (it's 1 for cars)
        if preference_cars:
            order_to_process_detections = [detection.id for detection in sorted(self.detections, key=attrgetter("class_no", "detection_score_sort"))]
        else:
            order_to_process_detections = [detection.id for detection in sorted(self.detections, key=attrgetter("class_no"))]
        # then we perform the process of deactivating overlapping boxes
        
        for detection_id in order_to_process_detections:
            #take the detection from the list of detections
            detection = self.detections[detection_id]
            #if it's active (i.e. those with higher scores haven't deactived it)
            if detection.active:
                #idenify possible overlaps
                detection.identify_possible_overlaps(all_detections_in_image = self.detections, num_quasi_radii = num_quasi_radii)
                #identify overlaps based on the 
                detection.identify_overlaps(all_detections_in_image = self.detections, intersection_over_union_cutoff = intersection_over_union_cutoff)
                
                #set these detections to inactive
                for overlapping_id in detection.overlaps:
                    self.detections[overlapping_id].active = False
    
    def plot_active_detections_cv2(self, circle_radius_ratio, circle_brg_colour_tuple, size):
        # plot the full image with small cicles over the active detection objects
        self.copy_for_plotting = copy.deepcopy(self.image_as_nparray)
        fig, ax = subplots(figsize=size)
        #plot the circles
        for detection in self.detections:
            if detection.active:
                cv2.circle(img = self.copy_for_plotting, center = (int(detection.centroidx), 
                                                                    int(detection.centroidy)), 
                                                                    radius = int(detection.quasi_radius*circle_radius_ratio), 
                                                                    color = circle_brg_colour_tuple,
                                                                    thickness=-1)
        ax.imshow(cv2.cvtColor(self.copy_for_plotting, cv2.COLOR_BGR2RGB), interpolation='nearest')
    
    def save_plotted_detections(self, save_location, circle_radius_ratio, circle_brg_colour_tuple):
        # plot the full image with small cicles over the active detection objects
        self.copy_for_plotting = copy.deepcopy(self.image_as_nparray)
        #plot the circles
        for detection in self.detections:
            if detection.active:
                cv2.circle(img = self.copy_for_plotting, center = (int(detection.centroidx), 
                                                                    int(detection.centroidy)), 
                                                                    radius = int(detection.quasi_radius*circle_radius_ratio), 
                                                                    color = circle_brg_colour_tuple,
                                                                    thickness=-1)
        cv2.imwrite(filename = save_location, img = self.copy_for_plotting)
    
    def number_of_cars(self):
        return len([item for item in self.detections if item.active])
    
    def average_box_radius(self):
        radii = []
        for detection in self.detections:
            if detection.active:
                radii.append(detection.quasi_radius)
        return sum(radii)/len(radii)
    
    def median_box_radius(self):
        radii = []
        for detection in self.detections:
            if detection.active:
                radii.append(detection.quasi_radius)
        return median(radii)
    
    def median_box_radius_above_score_percentile(self, percentile):
        radii = []
        for detection in self.detections:
            if detection.active:
                radii.append(detection.quasi_radius)
        return np.percentile(radii, percentile)
        
print("Classes written correctly")

In [15]:
import pickle
import pandas as pd
input_directory = r'C:\Users\paperspace\Documents\02 Imagery Collection'
example_plots_directory = r'C:\Users\paperspace\Documents\05 Inference (Car Counting)'
pickled_store_info_dir = 'image_reference.pickle'
machine_no = 1

output_data = []

store_info_list = pickle.load(open(os.path.join(input_directory,pickled_store_info_dir),'rb'))
relevant_rows = [row for row in store_info_list if row["machine_no"]==machine_no]
no_rows = len(relevant_rows)
for i, row in enumerate(relevant_rows):
    image_cache = image(row["directory"])
    image_cache.split_into_snippets(512, True)
    image_cache.detect_objects(detection_function=detect_fn, score_cuttoff=0.3)
    image_cache.remove_overlapping_objects(num_quasi_radii = 2, intersection_over_union_cutoff = 0.35, preference_cars = True)
    no_cars = image_cache.number_of_cars()
    row["no_cars"] = no_cars
    output_data.append(row)
    if i%10==0:
        #every 10 images print the progress, plot an example and try to write the data
        print("{}% complete".format(round(100*i/no_rows,2)))
        image_cache.save_plotted_detections(os.path.join(example_plots_directory,"image_{0}.jpg".format(row["id"])), 0.3, (255,0,0))
        try:
            pd.DataFrame(output_data).to_csv(os.path.join(example_plots_directory, "count_data.csv"))
        except:
            print("close the count_data.csv file")

0.0% complete
0.21% complete
0.42% complete
0.62% complete
0.83% complete
1.04% complete
1.25% complete
1.46% complete
1.67% complete
1.87% complete
2.08% complete
2.29% complete
2.5% complete
2.71% complete
2.92% complete
3.12% complete
3.33% complete
3.54% complete
3.75% complete
3.96% complete
4.16% complete
4.37% complete
4.58% complete
4.79% complete
5.0% complete
5.21% complete
5.41% complete
5.62% complete
5.83% complete
6.04% complete
6.25% complete
6.46% complete
6.66% complete
6.87% complete
7.08% complete
7.29% complete
7.5% complete
7.71% complete
7.91% complete
8.12% complete
8.33% complete
8.54% complete
8.75% complete
8.95% complete
9.16% complete
9.37% complete
9.58% complete
9.79% complete
10.0% complete
10.2% complete
10.41% complete
10.62% complete
10.83% complete
11.04% complete
11.25% complete
11.45% complete
11.66% complete
11.87% complete
12.08% complete
12.29% complete
12.49% complete
12.7% complete
12.91% complete
13.12% complete
13.33% complete
13.54% complete

# Getting a random evaluation sample

In [27]:
from numpy import random
import cv2
import os

evaluation_dataet_size = 30
storage_location = r"C:\Users\paperspace\Documents\Evaluation_Set"

random_indicies = random.randint(low=0, high=len(all_images_dirs)-1, size=evaluation_dataet_size)

chosen_image_directories = [all_images_dirs[item] for item in random_indicies]

for image_directory in chosen_image_directories:
    image_name = image_directory.split("\\")[-1]
    image = cv2.imread(image_directory)
    cv2.imwrite(os.path.join(storage_location, image_name), image)

In [17]:
pd.DataFrame(output_data).to_csv(os.path.join(example_plots_directory, "count_data_final_machine1.csv"),index=False)